In [91]:
import pandas as pd
import numpy as np
from IPython.display import display

In [92]:
file_path = r'C:\Users\serik\OneDrive\Рабочий стол\MLFINAL\AIRLINE PASSENGER SATISFACTION\train.csv'
df = pd.read_csv(file_path)

In [93]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [94]:
df.isnull().sum()

unnamed:_0                             0
id                                     0
gender                                 0
customer_type                          0
age                                    0
type_of_travel                         0
class                                  0
flight_distance                        0
inflight_wifi_service                  0
departure/arrival_time_convenient      0
ease_of_online_booking                 0
gate_location                          0
food_and_drink                         0
online_boarding                        0
seat_comfort                           0
inflight_entertainment                 0
on-board_service                       0
leg_room_service                       0
baggage_handling                       0
checkin_service                        0
inflight_service                       0
cleanliness                            0
departure_delay_in_minutes             0
arrival_delay_in_minutes             310
satisfaction    

In [95]:
df.dtypes

unnamed:_0                             int64
id                                     int64
gender                                object
customer_type                         object
age                                    int64
type_of_travel                        object
class                                 object
flight_distance                        int64
inflight_wifi_service                  int64
departure/arrival_time_convenient      int64
ease_of_online_booking                 int64
gate_location                          int64
food_and_drink                         int64
online_boarding                        int64
seat_comfort                           int64
inflight_entertainment                 int64
on-board_service                       int64
leg_room_service                       int64
baggage_handling                       int64
checkin_service                        int64
inflight_service                       int64
cleanliness                            int64
departure_

In [96]:
display(df)

,unnamed:_0,id,gender,customer_type,age,type_of_travel,class,flight_distance,inflight_wifi_service,departure/arrival_time_convenient,...,inflight_entertainment,on-board_service,leg_room_service,baggage_handling,checkin_service,inflight_service,cleanliness,departure_delay_in_minutes,arrival_delay_in_minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103899,103899,94171,Female,disloyal Customer,23,Business travel,Eco,192,2,1,...,2,3,1,4,2,3,2,3,0.0,neutral or dissatisfied
103900,103900,73097,Male,Loyal Customer,49,Business travel,Business,2347,4,4,...,5,5,5,5,5,5,4,0,0.0,satisfied
103901,103901,68825,Male,disloyal Customer,30,Business travel,Business,1995,1,1,...,4,3,2,4,5,5,4,7,14.0,neutral or dissatisfied
103902,103902,54173,Female,disloyal Customer,22,Business travel,Eco,1000,1,1,...,1,4,5,1,5,4,1,0,0.0,neutral or dissatisfied


In [97]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')

In [98]:
df['arrival_delay_in_minutes'] = imputer.fit_transform(df[['arrival_delay_in_minutes']])

In [99]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

In [100]:
df['age'] = scaler.fit_transform(df[['age']])
df['flight_distance'] = scaler.fit_transform(df[['flight_distance']])
df['arrival_delay_in_minutes'] = scaler.fit_transform(df[['arrival_delay_in_minutes']])
df['departure_delay_in_minutes'] = scaler.fit_transform(df[['departure_delay_in_minutes']])

In [101]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
one_hot = OneHotEncoder()
ordinal = OrdinalEncoder()

In [102]:
ohe_features = ['gender']
oe_features = ['customer_type', 'type_of_travel', 'class']

for feature in ohe_features:
    one_hot_df = pd.DataFrame(one_hot.fit_transform(df[[feature]]).toarray(), 
                              columns=one_hot.get_feature_names_out([feature]))
    df = df.drop(feature, axis=1).join(one_hot_df)
    
for feature in oe_features:
    df[feature] = ordinal.fit_transform(df[[feature]])

In [103]:
X = df.drop(["unnamed:_0", "id","satisfaction"], axis=1)
y = df["satisfaction"]

In [104]:
from sklearn.preprocessing import OneHotEncoder

ohe_label = OneHotEncoder()
y_encoded = ohe_label.fit_transform(y.values.reshape(-1, 1))
ohe_new_columns = list(ohe_label.get_feature_names_out(["satisfaction"]))
y_encoded_df = pd.DataFrame(y_encoded.toarray(), columns=ohe_new_columns, index=df.index)

In [105]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded_df, test_size=0.20, shuffle=True, random_state=1, stratify=y_encoded_df)

In [106]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler

In [107]:
def create_model(input_shape, dropout_rate=0.2, weight_decay=0.0001):
    model = Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))
    model.add(layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(2, activation='sigmoid'))
    return model

In [108]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * 0.1

In [113]:
input_shape = X_train.shape[1]
model = create_model(input_shape)
model.compile(optimizer=Adam(learning_rate=0.001), loss=BinaryCrossentropy(label_smoothing=0.1), metrics=['accuracy'])

callbacks = [LearningRateScheduler(scheduler, verbose=1)]
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=callbacks)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/50
2598/2598 [==============================] - 4s 1ms/step - loss: 0.4148 - accuracy: 0.8791 - val_loss: 0.3206 - val_accuracy: 0.9342 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/50
2598/2598 [==============================] - 4s 1ms/step - loss: 0.3408 - accuracy: 0.9258 - val_loss: 0.3148 - val_accuracy: 0.9343 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/50
2598/2598 [==============================] - 3s 1ms/step - loss: 0.3259 - accuracy: 0.9327 - val_loss: 0.2963 - val_accuracy: 0.9466 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/50
2598/2598 [==============================] - 3s 1ms/step - loss: 0.3185 - accuracy: 0.9359 - val_loss: 0.2954 - val_accuracy: 0.9467 - lr: 0.0010

Epoch 5: LearningRateScheduler setting learning rat

**XGboost below**

In [115]:
import xgboost as xgb

In [116]:
model = xgb.XGBClassifier(random_state=1)

In [117]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [6, 8, 10],
    'learning_rate': [0.001, 0.01, 0.1],
}

In [118]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=1, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.01, 0.1],
                         'max_depth': [6, 8, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy', verbose=1)

In [119]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation accuracy of the XGboost model: {accuracy:.2f}")

Validation accuracy of the XGboost model: 0.96


In [114]:
_, val_acc = model.evaluate(X_val, y_val, verbose=2)
print("Validation Accuracy of Keras: ", val_acc)

650/650 - 0s - loss: 0.2743 - accuracy: 0.9589 - 372ms/epoch - 572us/step
Validation Accuracy of Keras:  0.9588566422462463
